In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
from sklearn import linear_model
from tqdm.notebook import tqdm
import scipy.optimize as opt
import random

In [ ]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [ ]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [ ]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [ ]:
answers = {}

# Q1)

In [ ]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)
# t = []
# for i in allRatings:
#   if i[2] == 5:
#     t.append(i)
# print(len(t))

In [ ]:
ratingsTrain = allRatings
ratingsValid = allRatings
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [ ]:
(allRatings[:10])

[('u67805239', 'b61372131', 4),
 ('u54531895', 'b75189008', 4),
 ('u76549666', 'b75446982', 4),
 ('u03186275', 'b23482469', 2),
 ('u21322233', 'b09979253', 3),
 ('u00402241', 'b68456479', 1),
 ('u88999268', 'b49553867', 0),
 ('u39455611', 'b40151793', 5),
 ('u90502882', 'b01672704', 4),
 ('u92679832', 'b26246971', 4)]

In [ ]:
type(ratingsValid[:10])

list

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
# for i in tqdm(ratingsValid):
#   time.sleep(.001)
#   pass

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated

In [ ]:
ratingDict = {} # To retrieve a rating for a specific user/item pair

In [ ]:
for i in ratingsTrain:
  # if i[0] not in itemsPerUser:
  #   itemsPerUser[i[0]] = [i[1]]
  # else:
  itemsPerUser[i[0]].add(((i[1],1)))
  
  # if i[1] not in usersPerItem:
  #   usersPerItem[i[1]] = [i[0]]
  # else:
  usersPerItem[i[1]].add((i[0],1))

  ratingDict[(i[0],i[1])] = i[2]

  # print(itemsPerUser)
  

In [ ]:
# print(itemsPerUser)
print(max(len(itemsPerUser[x]) for x in itemsPerUser))

88


In [ ]:
n= len(ratingsValid)
allBooks = set(usersPerItem.keys())
for i in tqdm(range(n)):
  # creating non read set
  notRead = allBooks - set(itemsPerUser[ratingsValid[i][0]])
  randi = random.sample(notRead,1)
  
  # adding negative Samples
  ratingsValid.append((ratingsValid[i][0], randi[0], -1))
  

  0%|          | 0/200000 [00:00<?, ?it/s]

In [ ]:
cntBooksPop = defaultdict(int)
for i in tqdm(itemsPerUser):
  # print(itemsPerUser[i])
  for j in itemsPerUser[i]:
    cntBooksPop[j[0]]+=1
  
cntBooksPop = dict(sorted(cntBooksPop.items(), key=lambda item: item[1],reverse=True))
print(cntBooksPop)

  0%|          | 0/27945 [00:00<?, ?it/s]

{'b93745301': 974, 'b06167371': 889, 'b70912031': 864, 'b79928088': 838, 'b29802159': 750, 'b19867465': 716, 'b08004121': 704, 'b78275112': 702, 'b82148195': 696, 'b74821603': 666, 'b09049778': 664, 'b26138336': 645, 'b52929604': 620, 'b66664015': 617, 'b23760358': 607, 'b79253970': 602, 'b08545824': 596, 'b05154189': 589, 'b18610759': 569, 'b26385014': 519, 'b86788370': 517, 'b93997659': 507, 'b08349503': 502, 'b07278059': 499, 'b88947899': 491, 'b58973293': 487, 'b29576446': 483, 'b60484296': 479, 'b61372131': 463, 'b21674268': 461, 'b05231805': 458, 'b98844131': 455, 'b40598851': 454, 'b63603925': 446, 'b29738200': 446, 'b52736396': 443, 'b31687952': 439, 'b07812624': 437, 'b66908318': 432, 'b31988440': 430, 'b56105038': 421, 'b78008834': 418, 'b03946791': 415, 'b31362596': 410, 'b29998162': 410, 'b52842739': 395, 'b50384790': 395, 'b19822827': 386, 'b21557753': 378, 'b67584295': 373, 'b35526856': 372, 'b12526023': 367, 'b17187529': 358, 'b83439687': 357, 'b76040943': 356, 'b6998943

In [ ]:
# cntBooksPop["b69684851"] +=1

In [ ]:
totalReadings = sum(cntBooksPop.values())
print(totalReadings)
print(len(cntBooksPop.values()))

200000
6688


In [ ]:
cnt = 0
top1Per = set()
for i in cntBooksPop:
  top1Per.add(i)
  cnt+=cntBooksPop[i]
  if cnt>totalReadings//2:
    break
len(top1Per)


512

In [ ]:
# Evaluating
# it is mentioned that the baseline models returns whether a book has been read or not based on most pop
# print(ratingsValid[10000])
corr_pred = 0
for i in tqdm(ratingsValid):
  if i[1] in top1Per:
    if i[2]>=0:
      corr_pred+=1
  else:
    if i[2]==-1:
      corr_pred+=1
acc = corr_pred/len(ratingsValid)
print("Accuracy = ",corr_pred/len(ratingsValid)*100,"%")
answers["Q1"] = acc
print(answers)

  

  0%|          | 0/400000 [00:00<?, ?it/s]

Accuracy =  71.16075 %
{'Q1': 0.7116075}


# Q2)


In [ ]:
accuracy = {}
for k in tqdm(range(1,10001)):
  # print(type(i))
  # print(totalReadings)
  cnt = 0
  top1Per = set()
  for i in cntBooksPop:
    top1Per.add(i)
    cnt+=cntBooksPop[i]
    if cnt > (k*totalReadings)//10000:
      break
  corr_pred = 0
  for i in (ratingsValid):
    if i[1] in top1Per:
      if i[2]>=0:
        corr_pred+=1
    else:
      if i[2]==-1:
        corr_pred+=1
  acc = corr_pred/len(ratingsValid)
  accuracy[k] = acc


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
maxii = max(accuracy.values())
print(maxii)

0.7555775


In [ ]:
# print((accuracy.index(max(accuracy))+1),"\b% of the popular books")
key_list = list(accuracy.keys())
val_list = list(accuracy.values())
 
# print key with val 100
position = val_list.index(maxii)
print(key_list[position]/10000)
tmpii = key_list[position]/10000

0.738


In [ ]:
answers["Q2"] = (tmpii,maxii)

In [ ]:
answers

{'Q1': 0.7116075, 'Q2': (0.74, 0.755495)}

# Q3)

In [ ]:
def Jaccard(s1, s2):
    try:
      return len(s1.intersection(s2))/len(s1.union(s2))
    except:
      return 0

In [ ]:
accuracy = []
for thres in tqdm(range(1,101)):
  corr_pred = 0
  for i in (ratingsValid):
    # print(i)
    # break
    user = i[0]
    bpred = i[1]
    # print(user,bpred)
    # break
    jaccardd = []
    s1 = usersPerItem[bpred]
    s1.discard(user)
    for j in itemsPerUser[user]:
      
      s2 = usersPerItem[j[0]]
      # print(s1)
      # print(j[0])
      
      s2.discard(user)
      
      # break
      
      jaccardd.append(Jaccard(s1,s2))
    # break
    read = 0
    if jaccardd!=[] and max(jaccardd)>thres/100:
      read=1
    
    if read == 1 and i[2]>=0:
      corr_pred+=1
    elif read == 0 and i[2]==-1:
      corr_pred+=1

  acc = corr_pred/len(ratingsValid)
  accuracy.append(acc)

print(max(accuracy))
print((accuracy.index(max(accuracy))+1),"%")


  0%|          | 0/100 [00:00<?, ?it/s]

0.3049
1 %


In [ ]:
answers["Q3"] = max(accuracy)

In [ ]:
answers

{'Q1': 0.7052,
 'Q2': (0.74, 0.74505),
 'Q10': ['u19874911', 'u56426501', 1.8303186192979495, -3.5546973721391724],
 'Q9': 1.4344250687643854,
 'Q11': [4.46122830200134, 1.3808570184979263],
 'Q3': 0.3049}

# Q4)

In [ ]:
combiAcc = []

## using old optimization values

In [ ]:
# accuracy = []
corr_pred = 0
for i in tqdm(ratingsValid):
  # print(i)
  # break
  user = i[0]
  bpred = i[1]
  # print(user,bpred)
  # break
  jaccardd = []
  s1 = usersPerItem[bpred]
  s1.discard(user)
  for j in itemsPerUser[user]:
    
    s2 = usersPerItem[j[0]]
    # print(s1)
    # print(j[0])
    
    s2.discard(user)
    
    # break
    
    jaccardd.append(Jaccard(s1,s2))

    cnt = 0
    top1Per = set()
    for k in cntBooksPop:
      top1Per.add(k)
      cnt+=cntBooksPop[k]
      if cnt > (74*totalReadings)//100:
        break
  # break
  read = 0
  if jaccardd!=[] and max(jaccardd)>1/100 and bpred in top1Per:
    read=1
  
  if read == 1 and i[2]>=0:
    corr_pred+=1
  elif read == 0 and i[2]==-1:
    corr_pred+=1

acc = corr_pred/len(ratingsValid)
# accuracy.append(acc)

print(acc)
combiAcc.append(acc)
# print((accuracy.index(max(accuracy))+1),"%")


  0%|          | 0/10000 [00:00<?, ?it/s]

0.2569


## optimizing jaccard only

In [ ]:
accuracy = []
for thres in tqdm(range(1,101)):
  corr_pred = 0
  cnt = 0
  top1Per = set()
  for k in cntBooksPop:
    top1Per.add(k)
    cnt+=cntBooksPop[k]
    if cnt > (74*totalReadings)//100:
      break
  for i in (ratingsValid):
    # print(i)
    # break
    user = i[0]
    bpred = i[1]
    # print(user,bpred)
    # break
    jaccardd = []
    for j in itemsPerUser[user]:
      s1 = usersPerItem[bpred]
      s2 = usersPerItem[j[0]]
      # print(s1)
      # print(j[0])
      s1.discard(user)
      s2.discard(user)
      
      # break
      
      jaccardd.append(Jaccard(s1,s2))
    # break

    

    read = 0
    if jaccardd!=[] and max(jaccardd)>thres/100 and bpred in top1Per:
      read=1
    
    if read == 1 and i[2]>0:
      corr_pred+=1
    elif read == 0 and i[2]==0:
      corr_pred+=1

  acc = corr_pred/len(ratingsValid)
  accuracy.append(acc)

print(max(accuracy))
print((accuracy.index(max(accuracy))+1),"%")
combiAcc.append(max(accuracy))


## optimizing baseline only

In [ ]:
accuracy = []
for thres in tqdm(range(1,101)):
  corr_pred = 0
  cnt = 0
  top1Per = set()
  for k in cntBooksPop:
    top1Per.add(k)
    cnt+=cntBooksPop[k]
    if cnt > (thres*totalReadings)//100:
      break
  for i in (ratingsValid):
    # print(i)
    # break
    user = i[0]
    bpred = i[1]
    # print(user,bpred)
    # break
    jaccardd = []
    for j in itemsPerUser[user]:
      s1 = usersPerItem[bpred]
      s2 = usersPerItem[j[0]]
      # print(s1)
      # print(j[0])
      s1.discard(user)
      s2.discard(user)
      
      # break
      
      jaccardd.append(Jaccard(s1,s2))
    # break

    

    read = 0
    if jaccardd!=[] and max(jaccardd)>(1/100) and (bpred in top1Per):
      read=1
    
    if read == 1 and i[2]>0:
      corr_pred+=1
    elif read == 0 and i[2]==0:
      corr_pred+=1

  acc = corr_pred/len(ratingsValid)
  accuracy.append(acc)

print(max(accuracy))
print((accuracy.index(max(accuracy))+1),"%")


print(totalReadings)
combiAcc.append(max(accuracy))
  

  0%|          | 0/100 [00:00<?, ?it/s]

0.61985
77 %
190000


In [ ]:
# accuracy = []
corr_pred = 0
cnt = 0
top1Per = set()
for k in cntBooksPop:
  top1Per.add(k)
  cnt+=cntBooksPop[k]
  if cnt > (77*totalReadings)//100:
    break
for i in tqdm(ratingsValid):
  # print(i)
  # break
  user = i[0]
  bpred = i[1]
  # print(user,bpred)
  # break
  jaccardd = []
  for j in itemsPerUser[user]:
    s1 = usersPerItem[bpred]
    s2 = usersPerItem[j[0]]
    # print(s1)
    # print(j[0])
    s1.discard(user)
    s2.discard(user)
    
    # break
    
    jaccardd.append(Jaccard(s1,s2))

    
  # break
  read = 0
  if jaccardd!=[] and max(jaccardd)>1/100 and bpred in top1Per:
    read=1
  
  if read == 1 and i[2]>0:
    corr_pred+=1
  elif read == 0 and i[2]==0:
    corr_pred+=1

acc = corr_pred/len(ratingsValid)
# accuracy.append(acc)

print(acc)
# print((accuracy.index(max(accuracy))+1),"%")
combiAcc.append(acc)


In [ ]:
answers["Q4"]=max(combiAcc)
print(answers)

# Q5)

In [ ]:
import csv
# accuracy = []
testData = []
with open('/content/pairs_Read.csv', newline='') as csvfile:
  f = csv.reader(csvfile, delimiter=' ', quotechar='|')
  next(f)
  for i in f:
    testData.append(i[0].split(','))
    # break

print(testData[0])
# corr_pred = 0
cnt = 0
top1Per = set()
for k in cntBooksPop:
  top1Per.add(k)
  cnt+=cntBooksPop[k]
  if cnt > (74*totalReadings)//100:
    break
with open("Q5_pred.txt",'w',encoding = 'utf-8') as f:
  for i in tqdm(testData):
    # print(i)
    # break
    user = i[0]
    bpred = i[1]
    # print(user,bpred)
    # break
    jaccardd = []
    for j in itemsPerUser[user]:
      s1 = usersPerItem[bpred]
      s2 = usersPerItem[j[0]]
      # print(s1)
      # print(j[0])
      s1.discard(user)
      s2.discard(user)
      
      # break
      
      jaccardd.append(Jaccard(s1,s2))

      
    # break
    read = 0
    if jaccardd!=[] and max(jaccardd)>1/100 and bpred in top1Per:
      read=1
    f.write(str((user,bpred,read))+"\n")
  #   if read == 1 and i[2]>0:
  #     corr_pred+=1
  #   elif read == 0 and i[2]==0:
  #     corr_pred+=1

  # acc = corr_pred/len(ratingsValid)
  # # accuracy.append(acc)

  # print(acc)
  # print((accuracy.index(max(accuracy))+1),"%")


['u37758667', 'b99713185']


  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
answers["Q5"]="I confirm that I have uploaded an assignment submission to gradescope"

In [ ]:
answers

{'Q1': 0.71345,
 'Q4': 0.62025,
 'Q5': 'I confirm that I have uploaded an assignment submission to gradescope'}

# Q9)

In [ ]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [ ]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [ ]:
(allRatings[:10])

[('u67805239', 'b61372131', 4),
 ('u54531895', 'b75189008', 4),
 ('u76549666', 'b75446982', 4),
 ('u03186275', 'b23482469', 2),
 ('u21322233', 'b09979253', 3),
 ('u00402241', 'b68456479', 1),
 ('u88999268', 'b49553867', 0),
 ('u39455611', 'b40151793', 5),
 ('u90502882', 'b01672704', 4),
 ('u92679832', 'b26246971', 4)]

In [ ]:
type(ratingsValid[:10])

list

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
# for i in tqdm(ratingsValid):
#   time.sleep(.001)
#   pass

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated

In [ ]:
ratingDict = {} # To retrieve a rating for a specific user/item pair

In [ ]:
for i in ratingsTrain:
  # if i[0] not in itemsPerUser:
  #   itemsPerUser[i[0]] = [i[1]]
  # else:
  itemsPerUser[i[0]].add(((i[1],1)))
  
  # if i[1] not in usersPerItem:
  #   usersPerItem[i[1]] = [i[0]]
  # else:
  usersPerItem[i[1]].add((i[0],1))

  ratingDict[(i[0],i[1])] = i[2]

  # print(itemsPerUser)
  

In [ ]:
globalAvg = sum(ratingDict.values())/len(ratingDict)

In [ ]:
# import scipy.optimize
ratingsTrain[0]
# Bu Bi Rui

('u67805239', 'b61372131', 4)

In [ ]:
alpha = globalAvg

In [ ]:
Bu = defaultdict(float)
for user in tqdm(itemsPerUser):
  # print("Ratings",len(ratingsPerUser[user]))
  # print(ratingsPerUser[user])
  ratiTmp = 0
  for i in ratingsPerUser[user]:
    ratiTmp+=i[1]
  Bu[user] = (ratiTmp/len(ratingsPerUser[user]))

print(Bu)

  0%|          | 0/27917 [00:00<?, ?it/s]

defaultdict(<class 'float'>, {'u67805239': 1.0555555555555556, 'u54531895': 4.0, 'u76549666': 3.6666666666666665, 'u03186275': 2.8, 'u21322233': 4.0, 'u00402241': 3.8, 'u88999268': 3.75, 'u39455611': 4.785714285714286, 'u90502882': 4.0, 'u92679832': 4.2, 'u88271768': 3.625, 'u42487937': 4.1, 'u50260163': 4.4, 'u76802936': 3.857142857142857, 'u53795404': 3.5, 'u63958609': 4.333333333333333, 'u22899611': 3.3333333333333335, 'u50844125': 5.0, 'u11614716': 3.8333333333333335, 'u09810497': 3.375, 'u73528797': 3.75, 'u71509810': 4.095238095238095, 'u81603581': 3.75, 'u28722937': 3.55, 'u62762345': 4.0, 'u45384323': 3.769230769230769, 'u10740755': 3.1666666666666665, 'u07727286': 4.5, 'u75880159': 3.857142857142857, 'u51641688': 4.333333333333333, 'u14305105': 4.5, 'u70301397': 3.3636363636363638, 'u57215448': 4.6, 'u50078422': 2.4285714285714284, 'u57334483': 3.8333333333333335, 'u67073389': 0.0, 'u19381726': 3.25, 'u21748536': 3.4285714285714284, 'u52380539': 4.25, 'u56631546': 3.3333333333

In [ ]:
Bi = defaultdict(float)
for item in tqdm(usersPerItem):
  # print("Ratings",len(ratingsPerUser[user]))
  # print(ratingsPerUser[user])
  ratiTmp = 0
  # print(ratingsPerItem[item])
  # break
  for i in ratingsPerItem[item]:
    ratiTmp+=i[1]
  Bi[item] = (ratiTmp/len(ratingsPerItem[item]))

print(Bi)

  0%|          | 0/6684 [00:00<?, ?it/s]

defaultdict(<class 'float'>, {'b61372131': 3.341860465116279, 'b75189008': 4.222222222222222, 'b75446982': 3.3076923076923075, 'b23482469': 3.6153846153846154, 'b09979253': 3.611111111111111, 'b68456479': 3.2533333333333334, 'b49553867': 3.739130434782609, 'b40151793': 3.928, 'b01672704': 3.933333333333333, 'b26246971': 3.658536585365854, 'b33391853': 3.915151515151515, 'b21231766': 4.040816326530612, 'b46823063': 4.125, 'b28543546': 3.6666666666666665, 'b47873173': 3.8333333333333335, 'b00608458': 3.625, 'b18469261': 4.054545454545455, 'b95822040': 3.5, 'b52892780': 3.4705882352941178, 'b58789127': 3.1052631578947367, 'b96123158': 3.1379310344827585, 'b22242111': 3.800995024875622, 'b42439444': 2.8, 'b41681847': 3.5714285714285716, 'b60484296': 3.5789473684210527, 'b83072744': 4.090909090909091, 'b12151882': 4.4035087719298245, 'b84341506': 3.3, 'b17585340': 3.504, 'b76981983': 3.65, 'b29452634': 3.1481481481481484, 'b29802159': 4.054929577464788, 'b40598851': 3.8337236533957846, 'b14

In [ ]:
alpha = globalAvg
lamb = 1
while True:
  # alpha
  palpha = alpha
  # print(Bu['u67805239'])
  numi = 0
  for i in ratingsTrain:
    user = i[0]
    item = i[1]
    rating = i[2]
    numi += rating-(Bu[user]+Bi[item])
  alpha = numi/len(ratingsTrain)

  # bu:
  
  for user in itemsPerUser: # gives me user names
    numi = 0
    for item in itemsPerUser[user]: # items of that uer
      item = item[0] # one item
      numi+=ratingDict[(user,item)]-alpha-Bi[item]
    deno = lamb+len(itemsPerUser[user])
    Bu[user] = numi/deno
  
  # bi:
  
  for item in usersPerItem:
    numi = 0
    for user in usersPerItem[item]:
      user = user[0]
      numi+=ratingDict[(user,item)]-alpha-Bu[user]
    deno = lamb+len(usersPerItem[item])
    Bi[item] = numi/deno
  # print(palpha,alpha)
  if round(palpha,5) == round(alpha,5):
    break
# print("final",palpha,alpha)

In [ ]:
# print(round(alpha,9))

-1.254e-06


In [ ]:
# accuracy = []
sumi = 0
for i in tqdm(ratingsValid):
  # print(i)
  # break
  user = i[0]
  bpred = i[1]
  rating = i[2]
  # print(user,bpred)
  # break
  # print(i)
  rating_pred = alpha+Bu[user]+Bi[bpred]

  sumi+=(rating_pred-rating)**2
  

MSE = sumi/len(ratingsValid)
# accuracy.append(acc)
print(MSE)
# print(acc)
# print((accuracy.index(max(accuracy))+1),"%")
answers["Q9"] = MSE

  0%|          | 0/10000 [00:00<?, ?it/s]

1.4344250687643854


In [ ]:
answers

{'Q1': 0.7052,
 'Q2': (0.74, 0.74505),
 'Q10': ['u19874911', 'u56426501', 2.12465405666886, -3.3353668756570256],
 'Q9': 1.4344250687643854}

# Q10)

In [ ]:
maxiU = max(Bu, key=Bu.get)
print(maxiU,Bu[maxiU])

u19874911 1.8303186192979495


In [ ]:
miniU = min(Bu, key=Bu.get)
print(miniU,Bu[miniU])

u56426501 -3.5546973721391724


In [ ]:
answers["Q10"] = [maxiU,miniU, Bu[maxiU], Bu[miniU]]

In [ ]:
answers

{'Q1': 0.7052,
 'Q2': (0.74, 0.74505),
 'Q10': ['u19874911', 'u56426501', 1.8303186192979495, -3.5546973721391724],
 'Q9': 1.4344250687643854}

# Q11)

In [ ]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [ ]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [ ]:
(allRatings[:10])

[('u67805239', 'b61372131', 4),
 ('u54531895', 'b75189008', 4),
 ('u76549666', 'b75446982', 4),
 ('u03186275', 'b23482469', 2),
 ('u21322233', 'b09979253', 3),
 ('u00402241', 'b68456479', 1),
 ('u88999268', 'b49553867', 0),
 ('u39455611', 'b40151793', 5),
 ('u90502882', 'b01672704', 4),
 ('u92679832', 'b26246971', 4)]

In [ ]:
type(ratingsValid[:10])

list

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
# for i in tqdm(ratingsValid):
#   time.sleep(.001)
#   pass

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated

In [ ]:
ratingDict = {} # To retrieve a rating for a specific user/item pair

In [ ]:
for i in ratingsTrain:
  # if i[0] not in itemsPerUser:
  #   itemsPerUser[i[0]] = [i[1]]
  # else:
  itemsPerUser[i[0]].add(((i[1],1)))
  
  # if i[1] not in usersPerItem:
  #   usersPerItem[i[1]] = [i[0]]
  # else:
  usersPerItem[i[1]].add((i[0],1))

  ratingDict[(i[0],i[1])] = i[2]

  # print(itemsPerUser)
  

In [ ]:
globalAvg = sum(ratingDict.values())/len(ratingDict)

In [ ]:

def modell(lamb):
  # lamb=1
  Bu = defaultdict(float)
  for user in itemsPerUser:
    # print("Ratings",len(ratingsPerUser[user]))
    # print(ratingsPerUser[user])
    ratiTmp = 0
    for i in ratingsPerUser[user]:
      ratiTmp+=i[1]
    Bu[user] = (ratiTmp/len(ratingsPerUser[user]))

  # print(Bu)


  # BI
  Bi = defaultdict(float)
  for item in usersPerItem:
    # print("Ratings",len(ratingsPerUser[user]))
    # print(ratingsPerUser[user])
    ratiTmp = 0
    # print(ratingsPerItem[item])
    # break
    for i in ratingsPerItem[item]:
      ratiTmp+=i[1]
    Bi[item] = (ratiTmp/len(ratingsPerItem[item]))

  # print(Bi)

  alpha = globalAvg
  # lamb = 1
  while True:
    # alpha
    palpha = alpha
    # print(Bu['u67805239'])
    numi = 0
    for i in ratingsTrain:
      user = i[0]
      item = i[1]
      rating = i[2]
      # print(rating,Bu[user],Bi[item])
      numi += rating-Bu[user]-Bi[item]
      # print(numi)
    # print("NUMI",numi)
    alpha = numi/len(ratingsTrain)
    # print("ALPHA",alpha)
    f=1
    # bu:
    
    for user in itemsPerUser: # gives me user names
      numi = 0
      for item in itemsPerUser[user]: # items of that uer
        item = item[0] # one item
        numi+=ratingDict[(user,item)]-alpha-Bi[item]
      deno = float(lamb)+len(itemsPerUser[user])
      # print(deno)
      Bu[user] = numi/deno
    
    # bi:
    
    for item in usersPerItem:
      numi = 0
      for user in usersPerItem[item]:
        user = user[0]
        numi+=ratingDict[(user,item)]-alpha-Bu[user]
      deno = float(lamb)+len(usersPerItem[item])
      Bi[item] = numi/deno
    # print(palpha,alpha)
    if round(palpha,2) == round(alpha,2):
      break
  # print("final",palpha,alpha)
  # accuracy = []

  # MSE
  sumi = 0
  for i in tqdm(ratingsValid):
    # print(i)
    # break
    user = i[0]
    bpred = i[1]
    rating = i[2]
    # print(user,bpred)
    # break
    # print(i)
    rating_pred = alpha+Bu[user]+Bi[bpred]

    sumi+=(rating_pred-rating)**2
    

  MSE = sumi/len(ratingsValid)
  print("MSE",MSE,float(lamb))
  return MSE
  # accuracy.append(acc)

  # print(acc)
  # print((accuracy.index(max(accuracy))+1),"%")


In [ ]:
# lamb = 1
# # print(round(1.245532,2))
# result = opt.minimize(modell,lamb)

  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.4344181008853025 1.0


  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.4344181000822358 1.0000000149011612


  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.4216748071218939 1.2694644778966904


  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.4216748065066078 1.2694644927978516


KeyboardInterrupt: ignored

In [ ]:
# print(result)

      fun: 1.4340725564508017
 hess_inv: array([[1]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 2
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([1.])


In [ ]:
lamb = 10**-5
# print(round(1.245532,2))
result = opt.minimize(modell,lamb)

  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.569933207575905 1e-05


  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.5699332041178142 1.0014901161193848e-05


  0%|          | 0/10000 [00:00<?, ?it/s]

...

MSE 1.3808583587000127 4.433704777865519


  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.3808570184979263 4.46122830200134


  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.3808570184978912 4.461228316902501


In [ ]:
print(result)

      fun: 1.3808570184979263
 hess_inv: array([[291.3824638]])
      jac: array([-2.35438347e-06])
  message: 'Optimization terminated successfully.'
     nfev: 22
      nit: 10
     njev: 11
   status: 0
  success: True
        x: array([4.4612283])


In [ ]:
type(result)

scipy.optimize.optimize.OptimizeResult

In [ ]:
answers["Q11"] = [result["x"][0],result["fun"]]

In [ ]:
answers

{'Q1': 0.7052,
 'Q2': (0.74, 0.74505),
 'Q10': ['u19874911', 'u56426501', 1.8303186192979495, -3.5546973721391724],
 'Q9': 1.4344250687643854,
 'Q11': [4.46122830200134, 1.3808570184979263]}

In [ ]:
modell(1)

  0%|          | 0/10000 [00:00<?, ?it/s]

MSE 1.4340725564508015 1.0


1.4340725564508015